# Installs and imports
We do a source install of hyrax and import the major libraries we will need, as well as define constants so all our database accesses are on the same dataset.

In [1]:
# You must git check out hyrax to ~/rubin-user/hyrax for this to work.
%pip install -q -e ~/rubin-user/hyrax 2>&1 | grep -vE 'WARNING: Error parsing dependencies of (lsst-|astshim|astro-)'
%pip install -q lsdb 2>&1 | grep -vE 'WARNING: Error parsing dependencies of (lsst-|astshim|astro-)'


Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [3]:
import lsdb
import hyrax
from lsst.daf.butler import Butler
import numpy as np
import astropy.units as u

butler_config = {
    "config": "/repo/main",
    "collections": "LSSTComCam/runs/DRP/DP1/v29_0_0_rc5/DM-49865",
}
sky_config = {
    "skymap": "lsst_cells_v1",
    "tract": 5063,
    "patch": 4,
}

# Route 1 -- Create a Hats catalog with objects of interest

In order that this is compatible with DP1 ComCam data, we will pick a tract/patch where there is a deep field, cone search in hats slightly smaller than the patch to avoid edge-of-patch/edge-of-tract cutouts which are not yet handled by LSSTCutout, and then save the catalog file

In [2]:
lsdb_config = {
    "path": "/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/object_lc",
    "margin_cache": "/sdf/data/rubin/shared/lsdb_commissioning/hats/v29_0_0_rc5/object_lc_5arcs",
    "columns": [
        "objectId",
        "coord_ra",
        "coord_dec",
        "shape_flag",
        "g_kronMag",
        "g_psfMag",
        "shape_xx",
        "shape_yy",
        "shape_xy",
    ],
}

In [2]:
# Find the tract/patch dimensions we want
butler = Butler.from_config(**butler_config)
skymap = butler.get("skyMap", sky_config)
tract = skymap[sky_config["tract"]]
patch = tract.getPatchInfo(sky_config["patch"])
wcs = patch.getWcs()
patch_bbox = patch.getInnerBBox()

sky_max = wcs.pixelToSky(patch_bbox.minX, patch_bbox.maxY)
sky_min = wcs.pixelToSky(patch_bbox.maxX, patch_bbox.minY)

ra_range = [sky_min.getLongitude().asDegrees(), sky_max.getLongitude().asDegrees()]
dec_range = [sky_min.getLatitude().asDegrees(), sky_max.getLatitude().asDegrees()]

# Query the catalog and save out the restricted catalog
catalog = lsdb.read_hats(**lsdb_config)
catalog = catalog.box_search(ra=ra_range, dec=dec_range)
catalog = catalog.query("g_psfMag > 20 & g_psfMag < 30")
catalog = catalog.query("shape_flag == False")

catalog._ddf["area_sqpx"] = np.pi * np.sqrt(
    2 * (catalog._ddf["shape_xx"] * catalog._ddf["shape_yy"] - catalog._ddf["shape_xy"] ** 2)
)
catalog = catalog.query("area_sqpx > 5")
res = catalog.compute()
catalog.to_hats(base_catalog_path="./hyrax_catalog", catalog_name="hyrax_catalog", overwrite="True")
# catalog.columns
res

,objectId,coord_ra,coord_dec,shape_flag,g_kronMag,g_psfMag,shape_xx,shape_yy,shape_xy,area_sqpx
_healpix_29,,,,,,,,,,
2528665042651534972,2132999781327330046,53.149835,-28.353579,False,26.862612,27.239223,5.770088,3.617737,-1.410515,19.307438
2528665044302456614,2132999781327330243,53.147271,-28.352482,False,24.923187,25.08605,7.937774,8.092479,-1.153888,35.237640
...,...,...,...,...,...,...,...,...,...,...
2528737149810927662,2226792521223120438,53.15675,-28.187247,False,24.898371,26.014616,5.029204,6.267774,-0.069132,24.942400
2528737151063122411,2226748540758023356,53.15478,-28.187289,False,23.721519,24.283817,14.342004,12.251507,2.401523,57.918537


## Setup Hyrax to use the hats catalog
Configure hyrax to use the hats catalog 

In [5]:
h = hyrax.Hyrax()
h.config["data_set"]["name"] = "LSSTDataset"
h.config["data_set"]["hats_catalog"] = "./hyrax_catalog/"
h.config["data_set"]["butler_repo"] = butler_config["config"]
h.config["data_set"]["butler_collection"] = butler_config["collections"]
h.config["data_set"]["skymap"] = sky_config["skymap"]
h.config["data_set"]["semi_width_deg"] = (20 * u.arcsec).to(u.deg).value
h.config["data_set"]["semi_height_deg"] = (20 * u.arcsec).to(u.deg).value

a = h.prepare()

[2025-04-23 00:17:09,220 hyrax:INFO] Runtime Config read from: /sdf/data/rubin/user/mtauraso/hyrax/src/hyrax/hyrax_default_config.toml


/home/m/mtauraso/.local/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
[2025-04-23 00:17:14,816 hyrax.prepare:INFO] Finished Prepare


In [10]:
a[3].shape

torch.Size([3, 200, 202])

# Route 2 -- Use Butler to Download Catalog 

In [5]:
butler = Butler(**butler_config)

Let's specify the columns we want to downloda

In [6]:
INCOLS = [
    "objectId",
    "coord_ra",
    "coord_dec",
]

Now, let's download the data using the butler. We use the `sky_config` we had specified earlier

In [10]:
object_table = butler.get("object", dataId=sky_config, parameters={"columns": INCOLS})

The butler returns an Astropy table

In [11]:
object_table

objectId,coord_ra,coord_dec
int64,float64,float64
2226730948571955231,53.84420370780718,-28.240337522632426
2226730948571955234,53.858131730585136,-28.238685401609477
2226730948571955235,53.85369296384201,-28.237595175872777
2226730948571955236,53.85497966971418,-28.23732718026425
2226730948571955237,53.85206728015377,-28.23398234737432
2226730948571955238,53.86660564441939,-28.233416496002043
2226730948571955240,53.86916441028787,-28.232749888804847
2226730948571955241,53.86609497158882,-28.232530683870277
2226730948571955242,53.86861432865251,-28.231339170865187


Let's save the first 10,000 images as a pickle file (faster i/o compared to fits)

In [15]:
import pickle

with open("./test_catalog_10k.pkl", "wb") as f:
    pickle.dump(object_table[:10000], f)

## Setup Hyrax to use the astropy catalog
Configure hyrax to use the astropy catalog. Note that instead of saving the astropy table as a pickle file, you can save it any format that is supported by Astropy

In [16]:
h = hyrax.Hyrax()
h.config["data_set"]["name"] = "LSSTDataset"
h.config["data_set"]["astropy_table"] = "./test_catalog_10k.pkl"
h.config["data_set"]["butler_repo"] = butler_config["config"]
h.config["data_set"]["butler_collection"] = butler_config["collections"]
h.config["data_set"]["skymap"] = sky_config["skymap"]
h.config["data_set"]["semi_width_deg"] = (20 * u.arcsec).to(u.deg).value
h.config["data_set"]["semi_height_deg"] = (20 * u.arcsec).to(u.deg).value

[2025-05-26 01:07:05,687 hyrax:INFO] Runtime Config read from: /sdf/data/rubin/user/aritrag/hyrax/src/hyrax/hyrax_default_config.toml


In [18]:
a = h.prepare()

[2025-05-26 01:09:23,954 hyrax.prepare:INFO] Finished Prepare


In [19]:
a[1].shape

torch.Size([3, 203, 199])